# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
#print(len(full_data_rows_list))
# list of event data rows
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modelling in Apache Cassandra

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Data Modelling

### Query 1
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- The data filters on sessionId and itemInSession, a composite key (sessionId + itemInSession) has to be used to model table.

#### Create Table

In [8]:
# Create Table music_library1

query = "CREATE TABLE IF NOT EXISTS songs_playlist_by_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### Insert Data

In [9]:
# Read file event_datafile_new.csv and insert data in Table songs_playlist_by_session
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_playlist_by_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Run Query to verify records

In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

query = "select artist, song, length from songs_playlist_by_session WHERE session_id=338 AND item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row. artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- The data filters on userid and sessionid we need a composite key (userid + sessionid). Also, we need to sort by itemInSession and user (first and last name) we need t add clustering column to sort data.

#### Create Table

In [11]:
# Create Table music_library2

query = "CREATE TABLE IF NOT EXISTS users_playlist "
query = query + "(user_id int, session_id int, item_in_session int, first_name text, last_name text, artist text, song text, PRIMARY KEY ((user_id, session_id), item_in_session, first_name, last_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert Data

In [12]:
# Read file event_datafile_new.csv and insert data in Table users_playlist
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_playlist (user_id, session_id, item_in_session, first_name, last_name, artist, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

#### Run Query to verify records

In [13]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
query = "select artist, song, first_name, last_name, item_in_session from users_playlist WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row. artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- Since we need to find users who listens to a particular song, we need to partiton on song and on userId.

#### Create Table

In [14]:
# Create Table popular_songs

query = "CREATE TABLE IF NOT EXISTS popular_songs "
query = query + "(user_id int, song text, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert Data

In [15]:
# Read file event_datafile_new.csv and insert data in Table popular_songs
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO popular_songs (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Run Query to verify records

In [16]:
## Query 3: Give me every user name (first and last) in my music app history
# who listened to the song 'All Hands Against His Own'
query = "select song, first_name, last_name from popular_songs WHERE song = 'All Hands Against His Own' "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "drop table songs_playlist_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table popular_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()